## MMN make epochs from hand cleaned data

This file takes as input the raw time series that have already been hand cleaned (i.e., bad channels and and epochs have been marked). This script is then epoching the raw data, excluding all bad segments. 

In [199]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

import numpy as np
import csv
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
from tqdm import tqdm 
from atpbar import atpbar
from datetime import datetime
import mne
from autoreject import AutoReject
from mne.preprocessing import ICA, corrmap, create_ecg_epochs, create_eog_epochs

from pyprep.find_noisy_channels import NoisyChannels
from mne_icalabel import label_components


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def raw_to_events(raw):
    event_id = {'-1':-1, '1':1, '-2':2, '2':2, '-3':-3, '3':3}
    events = mne.events_from_annotations(raw, event_id=event_id)[0]
    if len(events)==0:
        events = np.zeros((1,3))
        events = events.astype(int)
    return events 

In [54]:
# Suppress MNE output
#mne.set_log_level('WARNING')
mne.set_log_level('ERROR')
#mne.set_log_level('CRITICAL')
#mne.set_log_level('INFO')



## Make epochs by session

In [194]:
paths = glob.glob(f"../hand_cleaning/mmn/*.fif")
paths.sort()

for path in paths:
    file_name = path[21:-24]
    
    raw = mne.io.read_raw_fif(path, preload=True)
    events = raw_to_events(raw)    
    #events = events[20:,:] # to exclude the initial priming ones
    #events[:,0] = events[:,0] - 154 #0.6s * 256Hz 

    epochs = mne.Epochs(raw, events, tmin=-0.1, tmax=0.8, baseline=(-0.1, 0.0), preload=True, reject_by_annotation=True)
    #reject_criteria = dict(eeg=100e-6)
    #epochs.drop_bad(reject=reject_criteria)
    
    if len (epochs)<50: 
        print (f'too few samples: {path}')
        continue
    epochs.save(f'epochs/1_by_session/{file_name}_epo.fif', overwrite=True)

too few samples: ../hand_cleaning/mmn/sub7-day4-mindfulness_mmn-raw_phot-events.fif


## Apply AutoReject

In [203]:
paths = glob.glob('epochs/*')
paths.sort()
for path in paths: 
    basename = os.path.basename(path)
    epochs = mne.read_epochs(path)
    ar = AutoReject()
    epochs_clean = ar.fit_transform(epochs)
    epochs_clean.save(f'epochs_ar/{basename}', overwrite=True)

Running autoreject on ch_type=eeg




Estimated consensus=0.60 and n_interpolate=4
Running autoreject on ch_type=eeg




Estimated consensus=0.60 and n_interpolate=4


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/autoreject/utils.py:73: UserWarning: 1 channels are marked as bad. These will be ignored. If you want them to be considered by autoreject please remove them from epochs.info["bads"].
  warnings.warn(


Running autoreject on ch_type=eeg




Estimated consensus=1.00 and n_interpolate=30


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/autoreject/utils.py:73: UserWarning: 1 channels are marked as bad. These will be ignored. If you want them to be considered by autoreject please remove them from epochs.info["bads"].
  warnings.warn(


No bad epochs were found for your data. Returning a copy of the data you wanted to clean. Interpolation may have been done.
Running autoreject on ch_type=eeg


...

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/autoreject/utils.py:73: UserWarning: 3 channels are marked as bad. These will be ignored. If you want them to be considered by autoreject please remove them from epochs.info["bads"].
  warnings.warn(






Estimated consensus=0.70 and n_interpolate=4


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/autoreject/utils.py:73: UserWarning: 3 channels are marked as bad. These will be ignored. If you want them to be considered by autoreject please remove them from epochs.info["bads"].
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/autoreject/utils.py:73: UserWarning: 5 channels are marked as bad. These will be ignored. If you want them to be considered by autoreject please remove them from epochs.info["bads"].
  warnings.warn(


Running autoreject on ch_type=eeg




Estimated consensus=0.70 and n_interpolate=4


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/autoreject/utils.py:73: UserWarning: 5 channels are marked as bad. These will be ignored. If you want them to be considered by autoreject please remove them from epochs.info["bads"].
  warnings.warn(
